In [ ]:
Docker资源清理以及Ubuntu系统清理
# 操作风险由低到高

In [ ]:
一、Docker 专项清理

In [ ]:
1. 基础清理
# 删除所有已停止的容器
docker container prune

# 删除所有未被使用的镜像（悬空镜像）
docker image prune

# 删除未被使用的网络
docker network prune

# 删除所有未被使用的卷（重要数据可能存储在卷中，谨慎操作）
docker volume prune

# 综合清理（包含以上所有未使用对象）
docker system prune

In [ ]:
2. 深度清理
# 强制删除所有未被容器使用的镜像（包括被使用的镜像的旧版本）
docker image prune -a

# 清理构建缓存（适用于使用 BuildKit 的情况）
docker builder prune

# 删除超过 24 小时的容器
docker container prune --filter "until=24h"

# 删除所有未被任何容器引用的卷（二次确认无重要数据）
docker volume prune --force

In [ ]:
3. 针对性清理
# 查看 Docker 磁盘占用
docker system df

# 按占用空间排序镜像
docker images --format "{{.ID}}\t{{.Size}}\t{{.Repository}}" | sort -k 2 -h

# 删除指定镜像
docker rmi <镜像ID>

# 清理某容器相关资源
docker rm -v <容器ID>  # -v 同时删除关联卷

In [ ]:
二、Docker构建缓存清理

In [ ]:
1. 构建缓存机制
Docker 的构建缓存分为两种类型：
（1）传统构建器缓存：存储在 /var/lib/docker/overlay2 中的分层文件系统
（2）BuildKit 缓存：更先进的构建系统，支持缓存导出和复用

In [ ]:
2. 清理构建缓存
（1）查看缓存占用
# 查看 Docker 总体磁盘使用
docker system df
# 查看详细构建缓存（仅 BuildKit）
docker buildx du --verbose
（2）基础清理命令
# 清理所有未使用的构建缓存（包括传统和 BuildKit）
docker builder prune # 这个指令基本就够了
# 强制清理所有构建缓存（包括正在使用的）
docker builder prune --all --force
（3）针对传统构建器的清理
# 删除所有悬空构建缓存
docker system prune --filter "type=build-cache"
# 删除超过 24 小时的构建缓存
docker system prune --filter "until=24h"
（4）BuildKit 专用清理
# 清理无效的 BuildKit 缓存
docker buildx prune
# 清理所有 BuildKit 缓存（包括正在使用）
docker buildx prune --all
（5）单次构建缓存控制
# 禁用缓存构建
docker build --no-cache -t my-image .
# 指定缓存来源
docker build --cache-from=my-cache-image -t new-image .

In [ ]:
1. 基础清理
# 清理 apt 缓存
sudo apt clean

# 删除已卸载软件的残留配置
sudo apt autoclean

# 自动移除不再需要的依赖
sudo apt autoremove

# 删除旧内核（保留1-2个最新版本）
sudo apt purge $(dpkg -l | awk '/^ii linux-image-*/{print $2}' | grep -v $(uname -r))

In [ ]:
2. 日志清理
# 清空所有日志（谨慎操作）
sudo journalctl --vacuum-time=7d  # 保留7天日志
sudo find /var/log -type f -name "*.log" -exec truncate -s 0 {} \;

# 删除系统崩溃报告
sudo rm /var/crash/*

# 清理用户日志
rm ~/.cache/debug.log

In [ ]:
3. 深度清理
# 查找大文件（>100MB）
sudo find / -type f -size +100M -exec ls -lh {} \; | awk '{ print $9 ": " $5 }'

# 清理缩略图缓存
rm -r ~/.cache/thumbnails/*

# 清理临时文件
sudo rm -rf /tmp/*
sudo rm -rf /var/tmp/*

# 清理旧快照（如果使用 snapper）
sudo snapper list | grep timeline | awk '{print $1}' | xargs -n1 sudo snapper delete

In [ ]:
四、清理后维护建议

In [ ]:
1. 定期维护
# 设置每周自动清理
echo "docker system prune -af && apt autoremove -y" | sudo tee /etc/cron.weekly/cleanup
2. 空间监控
# 安装监控工具
sudo apt install htop
3. 存储优化
# 迁移 Docker 存储位置（如果 /var/lib/docker 所在分区空间不足）
sudo systemctl stop docker
sudo rsync -av /var/lib/docker /new-path/
sudo mv /var/lib/docker /var/lib/docker.bak
sudo ln -s /new-path/docker /var/lib/docker
sudo systemctl start docker

In [ ]:
五、注意事项

In [ ]:
1.危险操作禁区
不要删除 /lib, /usr, /bin 等系统目录
谨慎操作 rm -rf / 类命令（建议使用 trash-cli 替代 rm）
2.数据备份
# 重要数据备份示例
tar -czvf backup_$(date +%Y%m%d).tar.gz /path/to/important_data

In [ ]:
建议按顺序执行，每完成一个步骤后使用 df -h 查看空间变化。
如果空间仍未释放，可能需要检查是否有被进程占用的已删除文件（使用 lsof | grep deleted 查找）。